# Coursera Capstone Project

In [1]:
import pandas as pd
import numpy as np

In [2]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


In [68]:
from bs4 import BeautifulSoup
import requests

### Scrape the page

In [119]:
page_response = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M", timeout=5)
page_content = BeautifulSoup(page_response.content, "html.parser")
table = page_content.table
rows = table.tbody.contents

#remove the header and line break
rows.pop(0)

<tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>

### Iterate the rows and build dataframe

In [161]:
array = np.empty(shape=[0, 3])
for row in rows :
    try:
        columns = row.find_all("td")
        postCode = columns[0].text
        borough = columns[1].text
        neighborhood = columns[2].text.replace('\n','')
        #skip borough with "Not assigned" value
        if (borough == 'Not assigned') :
            continue
        array = np.append(array, [[postCode, borough, neighborhood]], axis=0)
    except:
        pass
    
df = pd.DataFrame(array)
df.columns = ['PostalCode', 'Borough', 'Neighborhood']
df = df.groupby(['PostalCode', 'Borough'])['Neighborhood'].agg(lambda col: ', '.join(col)).reset_index()

In [162]:
#deal with "Not assigned" neighborhood
def set_not_assigned_neighborhood(row):
    if (row['Neighborhood'] == 'Not assigned') :
        return row['Borough']
    
    return row['Neighborhood']

df['Neighborhood'] = df.apply(set_not_assigned_neighborhood, axis=1)

In [167]:
df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [169]:
df.shape

(103, 3)

### Build coordinates

In [177]:
df_coord = pd.read_csv('https://cocl.us/Geospatial_data')

In [178]:
df_coord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [181]:
df = df.join(df_coord.set_index('Postal Code'), on="PostalCode")

In [182]:
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
